<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-cad_kbcm_jfl/blob/reto/Reto_entrega_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instituto Tecnológico de Estudios Superiores de Monterrey**

**Maestría en Inteligencia Artificial Aplicada**

**Ciencia y Analítica de datos**

**Nombre del trabajo:** Reto -> Entrega 1: Limpieza, análisis, visualización y kmeans.

**Nombre del maestro:** María de la Paz Rico Fernández

**Nombre de los estudiantes:** Kevin Brandon Cruz Jorge Fernández Lara

**Matrículas:** A01794176 A01793062

**Fecha de entrega:** 16 de noviembre de 2022

**Objetivo:** Implementar conocimientos adquiridos a lo largo del curso en el desarrollo de un proyecto con datos reales.

Elige una de las bases de datos, ya sea la de las aguas superficiales o la de las aguas subterráneas.

Trataremos en este caso las aguas superficiales.

In [1]:

!pip install qeds fiona geopandas xgboost gensim folium pyLDAvis descartes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.7 MB 506 kB/s 
     |████████████████████████████████| 1.0 MB 38.9 MB/s 
     |████████████████████████████████| 1.7 MB 32.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 179 kB 63.4 MB/s 
     |████████████████████████████████| 6.3 MB 46.9 MB/s 
  Created wheel for qeds: filename=qeds-0.7.0-py3-none-any.whl size=27812 sha256=309790fa45b2e01dbc20e46b993cc73c621cc7b26f9811f215400d5cc335abc6
  Stored in directory: /root/.cache/pip/wheels/fc/8c/52/0cc036b9730b75850b9845770780f8d05ed08ff38a67cbaa29
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=834fa87e6b67d0771c4f10b6e6de36f0af94b7e7713df37729459e4360579e8a
  Stored in directory: /root/.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.parsers.readers import read_csv

In [3]:
url = 'https://github.com/PosgradoMNA/actividades-del-projecto-cad_kbcm_jfl/blob/main/Reto/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_superficiales_2020.csv'
df = pd.read_csv(url, sep=',')

ParserError: ignored